<a href="https://colab.research.google.com/github/Asadtheanalyst/Project/blob/main/python_book_web_scraping_plus_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries

from bs4 import BeautifulSoup
import requests
import time
import datetime

import smtplib

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List to hold all scraped data
data_list = []

# Base URL for the search results, you will need to update the query parameters as needed.
base_url = 'https://www.amazon.com/s?k=python+books'

# Headers to simulate a browser visit
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1"
}

# Loop through many pages you like
for page_number in range(1, 20):
    print(f"Scraping page {page_number}")

    # Construct the full URL with the current page number
    url = base_url + str(page_number)
    response = requests.get(url, headers=headers)

    # Check if the page was loaded successfully
    if response.status_code != 200:
        print(f"Failed to retrieve page {page_number}")
        continue

    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all product containers - this class name will likely be different and needs to be updated
    products = soup.findAll('div', class_='s-result-item')

    for product in products:
        # Initialize dictionary to hold product details
        product_details = {
            'Title': "Title not found",
            'Price': "Price not found",
            'Rating': "Rating not found",
            'Author': "Author not found"
        }

        # Extract product title
        title_tag = product.find('span', class_='a-size-base-plus a-color-base a-text-normal')
        if title_tag:
          product_details['Title'] = title_tag.get_text().strip()


        # Extract product price
        price_tag = product.find('span', class_='a-price-whole')
        price_fraction = product.find('span', class_='a-price-fraction')
        if price_tag:
            product_details['Price'] = price_tag.get_text().strip()
            if price_fraction:
                product_details['Price'] += '.' + price_fraction.get_text().strip()

        # Extract rating
        rating_tag = product.find('span', class_='a-icon-alt')
        if rating_tag:
            product_details['Rating'] = rating_tag.get_text().strip().split()[0]  # Assumes format "4.5 out of 5 stars"

        # Extract author
        author_tag = product.find('a', class_='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style')
        if author_tag:
          product_details['Author'] = author_tag.get_text().strip()

        # Append the product details to our list
        data_list.append(product_details)

# Create a DataFrame from our list of dictionaries
df = pd.DataFrame(data_list)

# Export the DataFrame to a CSV file
df.to_csv('amazon_books.csv', index=False)

print("Scraping completed and data exported to CSV.")


Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping completed and data exported to CSV.


In [20]:
df

,Title,Price,Rating,Author
0,Title not found,Price not found,4.5,Author not found
1,Title not found,Price not found,Rating not found,Author not found
2,Title not found,Price not found,Rating not found,Author not found
3,Title not found,29..99,4.6,John C. Shovic
4,Title not found,0..00,4.2,Author not found
...,...,...,...,...
525,Title not found,11..69,4.7,Max Wainewright
526,Title not found,Price not found,Rating not found,Author not found
527,Title not found,Price not found,Rating not found,Author not found
528,Title not found,Price not found,Rating not found,Author not found


In [8]:
print(df.describe())

# Descriptive statistics for categorical columns
print(df.describe(include='object'))

                  Title            Price            Rating            Author
count               530              530               530               530
unique                1              114                21                 1
top     Title not found  Price not found  Rating not found  Author not found
freq                530              185               198               530
                  Title            Price            Rating            Author
count               530              530               530               530
unique                1              114                21                 1
top     Title not found  Price not found  Rating not found  Author not found
freq                530              185               198               530


In [9]:
# Check the data types of all columns
print(df.dtypes)


Title     object
Price     object
Rating    object
Author    object
dtype: object


In [10]:
# Check for missing values
print(df.isnull().sum())


Title     0
Price     0
Rating    0
Author    0
dtype: int64


In [11]:
print(df.nunique())

Title       1
Price     114
Rating     21
Author      1
dtype: int64


In [12]:
# Check the distribution of categories for a specific column
print(df['Author'].value_counts())


Author not found    530
Name: Author, dtype: int64


In [13]:
print(df.corr())

Empty DataFrame
Columns: []
Index: []


<ipython-input-13-23236a4e6045>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.corr())


**We see here many title and other things are empty so now we goinf to use three loops instead of two**
which mean frist we go furst of pages, then for whole page product and then product


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# List to hold all scraped data
data_list = []

# Function to extract details from a product's page
def get_product_details(url, headers):
    product_details = {
        'Title': "Title not found",
        'Price': "Price not found",
        'Rating': "Rating not found",
        'Author': "Author not found"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return product_details

    product_soup = BeautifulSoup(response.content, 'html.parser')

    # Extract Title
    title_tag = product_soup.find('span', id='productTitle')
    if title_tag:
        product_details['Title'] = title_tag.get_text().strip()

    # Extract Price
    price_tag = product_soup.find('span', id='price')
    if price_tag:
        product_details['Price'] = price_tag.get_text().strip()

    # Extract Rating
    rating_tag = product_soup.find('span', class_='a-size-base a-color-base')
    if rating_tag:
    # This might contain "4.5 out of 5 stars"
       rating_text = rating_tag.get_text().strip()
    # Extract just the numeric part of the rating
       rating_value = rating_text.split()[0]
       product_details['Rating'] = rating_value


    # Extract Author
    author_tag = product_soup.find('span', class_='author notFaded')
    if author_tag:
        product_details['Author'] = author_tag.get_text().strip()

    return product_details

# Headers to simulate a browser visit
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1"
}

# Base URL for the search results
base_url = 'https://www.amazon.com/s?k=python+books&page='

# Loop through the first 20 pages of search results
for page_number in range(1, 20):
    print(f"Scraping page {page_number}")
    url = f"{base_url}{page_number}"
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve page {page_number}")
        continue

    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the links to individual product pages
    # The class or ID here is just an example; you need to use the correct one based on the actual web page
    product_links = soup.find_all('a', class_='a-link-normal s-no-outline')

    for link in product_links:
         href = link.get('href')
    # Check if 'href' already contains the full URL
         if not href.startswith('http'):
            href = 'https://www.amazon.com' + href
         product_details = get_product_details(href, headers)
         data_list.append(product_details)

    # A respectful delay between requests
         time.sleep(1)


# Convert list to DataFrame
df = pd.DataFrame(data_list)

# Export to CSV
df.to_csv('amazon_python_books_detailed.csv', index=False)

print("Scraping completed and data exported to CSV.")


due to mass running this again and again it block my ip but It would work if you follow and use good servics but that code is working and will work and create you a data set.